<a href="https://colab.research.google.com/github/Jenni01-12/Parcial-Final/blob/main/Parcial_final_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Titulo

# **Instalación de librerías**

In [ ]:
!pip install transformers datasets faiss-cpu sentence-transformers
!pip install transformers gradio --quiet
!pip install pypdf langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Cargar modelo preentrenado GPT-2
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Crear pipeline de generación de texto
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)


Device set to use cpu


In [ ]:
def responder(pregunta):
    prompt = f"Explícamelo como si fuera principiante en economía: {pregunta}\nRespuesta:"
    respuesta = chatbot(prompt, max_length=150, num_return_sequences=1, do_sample=True)[0]["generated_text"]
    return respuesta[len(prompt):].strip()


In [ ]:
import gradio as gr

interfaz = gr.Interface(fn=responder,
                        inputs="text",
                        outputs="text",
                        title="Chatbot de Economía para Dummies",
                        description="Haz cualquier pregunta de economía y obtén una respuesta sencilla")

interfaz.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5539a07f2769ab4aee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
